In [7]:
import os.path
import sys

import os

# Récupère le chemin d'accès parent de sys.path[0]
parent_dir = os.path.dirname(sys.path[0])

sys.path.append(os.path.abspath(parent_dir))
import importlib
import algorithmes
importlib.reload(algorithmes)
from algorithmes import *
import random
import mesa
import bisect

import gym
import numpy as np
import random
import tensorflow as tf
from collections import deque


In [8]:
nombre_client  = 10

STATE_SIZE = 6400
ACTION_SIZE = 3
LEARNING_RATE = 0.001
GAMMA = 0.99
BATCH_SIZE = 32
MEMORY_SIZE = 1000000


In [27]:
# Extraire tous les arcs de la route
route = [[1,2,3,4],[8,6,7],[5,9]]
arcs_voiture = set()
for sous_route in route:
    sous_route = [0]+sous_route+[0]
    arcs_voiture.update(set(zip(sous_route, sous_route[1:])))

# Initialiser la matrice d'adjacence avec des zéros
n = max(list(itertools.chain.from_iterable(route)))+1
matrice_adj = [[0]*n for _ in range(n)]

# Mettre à jour les valeurs correspondantes à chaque arc
for i, j in arcs_voiture:
    matrice_adj[i][j] = 1
    #matrice_adj[i][j] = matrice_temps_Cij[i][j]
     
    
print(np.array(matrice_adj))

[[0 1 0 0 0 1 0 0 1 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]]


In [28]:
def get_matrice_adjacence(route):
     
    # Extraire tous les arcs de la route
    arcs_voiture = set()
    for sous_route in route1:
        sous_route = [0]+sous_route+[0]
        arcs_voiture.update(set(zip(sous_route, sous_route[1:])))

    # Initialiser la matrice d'adjacence avec des zéros
    n = max(list(itertools.chain.from_iterable(route)))+1
    matrice_adj = [[0]*n for _ in range(n)]

    # Mettre à jour les valeurs correspondantes à chaque arc
    for i, j in arcs_voiture:
        matrice_adj[i][j] = 1
        #matrice_adj[i][j] = matrice_temps_Cij[i][j]
        
    
    return(np.array(matrice_adj))


In [9]:
# genere des solution aléatoire
def get_solution_random():
    list_client =  [i for i in range(nombre_client)]
    random.shuffle(list_client)
    solution = get_route_version2(list_client,time_window,Q)
    return(solution)

## utiliser fonction cout pour calculer le cout d'une fonction